In [ ]:
# default_exp scrape8K

# scrape8K

> Scrape item summaries from 8-K SEC filings.

In [ ]:
#hide
%load_ext autoreload
%autoreload 2
from nbdev import show_doc

In [ ]:
#export

import collections
import itertools
import os
import re

from secscan import utils, dailyList, basicInfo, infoScraper

default8KDir = os.path.join(utils.stockDataRoot,'scraped8K')

8K scraper class - scrape items summary from the SEC filing:

In [ ]:
#export

secItemHeaderPatStrs = [
    r'item[^a-z]*?(\d+(?:\.\d*)?)\.(?!\d)',
    r'item[^a-z]*?(\d+(?:\.\d*)?)(?=[a-z\s]*[.:])',
]
secItemPats = [re.compile(secItemHeaderPatStr
                            + r'\s*(.{80,}?)('+secItemHeaderPatStr+'|$)',
                            re.IGNORECASE)
                for secItemHeaderPatStr in secItemHeaderPatStrs]

def getSecItemsFromTextUsingPat(txt, secItemPat, filterF=None ,maxHeadLen=80, maxBodyLen=500) :
    pos = 0
    res = []
    while True :
        m = secItemPat.search(txt,pos)
        if m is None :
            return res
        g = m.groups()
        if len(res)>0 and res[-1][0] == g[0] :
            pos = m.start()+1
        else :
            head = g[1][:maxHeadLen]
            #if head.find('.') >= 0 :
            #    head = head[:head.find('.')+1]
            newTup = (g[0],head,g[1][:maxBodyLen])
            pos = m.start(3)
            if (filterF is None) or filterF(newTup) :
                res.append(newTup)

def getSecItemsFromText(txt, filterF=None, maxHeadLen=80, maxBodyLen=500) :
    """
    Returns a list of item summaries for items from 8-K and SC 13D forms:
        format in form:
            Item 1.01. WWW. YYY ZZZ
            Item 3.01. AAA. BBB CCC
        returned value:
            [('1.01', 'WWW.', 'WWW. YYY ZZZ')
             ('3.01', 'AAA.', 'AAA. BBB CCC') ... ]
    """
    for secItemPat in secItemPats :
        res = getSecItemsFromTextUsingPat(txt,secItemPat,filterF,maxHeadLen,maxBodyLen)
        if res :
            break
    return res

financialStatementsPat = re.compile(r'[^a-z]*financial\s*statements',
                                    re.IGNORECASE)
def getSec8KItemsFromText(txt) :
    """
    Returns list of item summaries for 8-K forms, ignoring the
    'Financial Statements and Exhibits' item.
    """
    return getSecItemsFromText(txt, lambda item : not financialStatementsPat.match(item[1]))

def parse8K(accessNo, formType=None) :
    info = basicInfo.getSecFormInfo(accessNo, formType)
    links = info['links']
    if len(links) == 0 :
        utils.printErrInfoOrAccessNo('NO LINKS LIST in',accessNo)
        return info
    # print(links[0])

    firstSLText = utils.getCombSoupText(utils.downloadSecUrl(links[0][3], toFormat='soup'))
    return info,firstSLText
#     print(firstSLText)
#     print 'size',len(firstSLText)
#     print utils.secUrlPref+links[0][3]
#     info['itemsdetail'] = getSec8KItemsFromText(firstSLText)
#     if info['itemsdetail'] == [] :
#         print('NO ITEMS:',utils.secUrlPref+links[0][3])
#     else :
#         print('.', #info['itemsdetail'],
#               end=' ')
#     return info

class scraper8K(infoScraper.scraperBase) :
    def __init__(self, infoDir=default8KDir, startD=None, endD=None, fSuff='m.pkl', **pickle_kwargs) :
        super().__init__(infoDir, '8-K', startD=startD, endD=endD, fSuff=fSuff, **pickle_kwargs)
    def scrapeInfo(self, accNo, formType=None) :
        return parse8K(accNo, formType), None

Test 8-K scraper class:

In [ ]:
# itemNoPat = re.compile(r'(item\s*[\d\.]*)',re.IGNORECASE)

['Item 1.02: Termination of a Material Definitive Agreement',
 'Item 3.02: Unregistered Sales of Equity Securities',
 'Item 8.01: Other Events',
 'Item 9.01: Financial Statements and Exhibits']

In [ ]:
#hide
# uncomment and run to regenerate all library Python files
# from nbdev.export import notebook2script; notebook2script()